[Altair Visualization](#Altair-visualization)

# Altair Visualization
<div style = "float:right"><a style="text-decoration:none" href = "#inicio">Inicio</a></div>

In [1]:
# Import Libraries
import altair as alt
# import streamlit as st
import pandas as pd

## Taylor series Data frames

### Dual Moving Average

In [2]:
# read csv
dma = pd.read_csv('../reports/dma_strategy.csv')

# rename first column as date
dma = dma.rename(columns={'Unnamed: 0':'date'})

# monthly resample
dma.date = pd.to_datetime(dma.date)
dma = dma.resample('M', on='date').mean().reset_index()

# rename columns of interest (stock prices and dual moving averages)
dma = dma.rename(columns={'price': '_AAPL', 'fast_ma': '_fast_ma', 'slow_ma': '_slow_ma'})

# melt stock prices and dual moving averages into 'series' new variable
keys = [x for x in dma.columns if x.startswith('_')]
dma_series = pd.melt(dma, id_vars=['date', 'portfolio_value'], value_vars=keys, value_name='price', var_name='series')

In [191]:
dma_series.head()

,date,portfolio_value,series,price
0,2015-05-31,10000.000000,_fast_ma,NaN
1,2015-06-30,10000.000000,_fast_ma,127.014375
2,2015-07-31,9938.030682,_fast_ma,125.907301
3,2015-08-31,9702.420500,_fast_ma,115.321565
4,2015-09-30,9662.803357,_fast_ma,112.563988


### ML Buy and Hold

In [3]:
# read csv
gb = pd.read_csv('../reports/buy_and_hold_GB.csv')

# rename first column as date
gb = gb.rename(columns={'Unnamed: 0':'date'})

# monthly resample
gb.date = pd.to_datetime(gb.date)
gb = gb.resample('M', on='date').mean().reset_index()

# rename columns of interest (stock prices and dual moving averages)
gb = gb.rename(columns={'AAPL': 'price'})


## Altair Graphs

In [4]:
select_time = alt.selection(type="interval",encodings=["x"])

portfolio_dma = alt.Chart(dma_series).mark_line().encode(
    x="date",
    y='portfolio_value'
).transform_filter(
    select_time
)

price_dma = alt.Chart(dma_series).mark_line().encode(
    x="date",
    y='price',
    color='series',
    strokeDash='series'
).properties(
    selection=select_time
)

portfolio_gb = alt.Chart(gb).mark_line().encode(
    x="date",
    y='portfolio_value'
).transform_filter(
    select_time
)

price_gb = alt.Chart(dma_series).mark_line().encode(
    x="date",
    y='price'
).properties(
    selection=select_time
)

(portfolio_dma&price_dma) | (portfolio_gb&price_gb)

alt.HConcatChart(...)